In [132]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

1: независиме

2: то где есть $\Delta$

3: стоит использовать мана уитни и $2^{n}$

    wrong: всё кроме $C_{2n}^{n}$

4: 0.2845
    
    wrong:0.1350; 0.0440, 0.0663

5: 0.0290

6: 1.423

7: 0.0057

# 4 Пациенты с лейкомией

In [133]:
X = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])
medX=200 

In [134]:
m0 = 0.5
stats.wilcoxon(X - medX)

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

# 5 Лесозаготовка

In [135]:
destoying = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
not_destr = np.array([17,18,18,15,12,4,14,15,10])

In [136]:
stats.mannwhitneyu(destoying, not_destr)

MannwhitneyuResult(statistic=27.0, pvalue=0.02900499272087373)

# 6 Challenger


В 6 и 7 заданиях следует считать "повреждениями" записи с Incident == 0, и наоборот.

In [137]:
data = pd.read_csv('challenger.txt', sep = '\t')

In [138]:
data.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [139]:
invalid = data[data['Incident']  == 1].Temperature.values
valid = data[data['Incident']  == 0].Temperature.values

In [140]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [141]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [142]:
np.random.seed(0)
booted_inv = map(np.mean, get_bootstrap_samples(valid, 1000))
booted_v = map(np.mean, get_bootstrap_samples(invalid, 1000))

diff_in_mean = list(map(lambda x: x[1] - x[0], zip(booted_v, booted_inv)))

In [143]:
print( "95% confidence interval for diff in temp:",  stat_intervals(diff_in_mean, 0.05))

95% confidence interval for diff in temp: [1.42299107 7.93861607]


# 7

In [144]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [145]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [146]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [147]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [148]:
np.random.seed(0)
print ("p-value: %f" % permutation_test(invalid, valid, max_permutations = 10000))

p-value: 0.005700
